# 잠재 요인 협업 필터링

## 잠재 요인 협업 필터링의 이해
- 행렬 분해(Matrix Factorization) : 대규모 다차원 행렬을 SVD와 같은 차원 감소 기법으로 분해하는 과정에서 잠재 요인을 추출하는 것
- 잠재 요인이 어떤 것인지 명확히 정의할 수는 없음
- 그러나 다차원 희소 행렬인 사용자-아이템 행렬 데이터를 저차원 밀집 행렬의 사용자-잠재 요인 행렬과 아이템-잠재 요인 행렬(의 전치행렬)로 분해할 수 있음
- 이렇게 분해된 두 행렬의 내적을 통해 새로운 예측 사용자-아이템 데이터를 만들어 사용자가 아직 평가하지 않은 아이템에 대한 예측 평점을 생성

## 행렬 분해의 이해
- SVD(Singular Vector Decomposition), NMF(Non-Negative Matrix Factorization) 등
- $R = PQ^T$
    - $R$ : $M$ x $N$ 의 사용자-아이템 평점 행렬
    - $P$ : $M$ x $K$ 의 사용자-잠재요인 행렬
    - $Q$ : $N$ x $K$ 의 아이템-잠재요인 행렬
    - $M$ : 총 사용자 수
    - $N$ : 총 아이템 수
    - $K$ : 잠재 요인의 차원 수
- $r_{(u, i)} = p_u q_i^t$
    - $r_{(u, i)}$ : u행 사용자의 i열 아이템에 대한 평점
    - $p_u$ : u행 사용자의 잠재요인
    - $q_i$ : i행 아이템의 잠재요인
- $R \cong \hat R = PQ^T$
    - $\hat R$ : 예측 사용자-아이템 평점 행렬
- 주로 SVD 이용
    - 그러나 Null이 있는 행렬에 대해선 불가능
    - 그렇기 때문에 확률적 경사 하강법(Stochastic Gradient Descent)이나 ALS(Alternating Least Squares) 방식을 이용하여 SVD 수행

## 확률적 경사 하강법을 이용한 행렬 분해
- 경사 하강법의 한 종류
- P와 Q로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q를 유추하는 것

### 확률적 경사 하강법 절차
1. P와 Q를 임의의 값을 가진 행렬로 설정
2. P와 Q.T를 곱해 예측 행렬 R을 계산, 실제 행렬 R과의 오차 계산
3. 이 오차를 최소화할 수 있도록 P와 Q 행렬 업데이트
4. 만족할 만한 오류 값을 가질 때까지 2, 3번 작업 반복하면서 P와 Q를 업데이트해 근사화
- 오차와 L2 규제를 고려한 비용 함수식  
$\min \sum \left( r_{(u, i)} - p_uq_i^t\right)^2 + \lambda \left(\lVert q_i \rVert^2 + \lVert p_u \rVert ^2 \right)$  
- 새롭게 업데이트 되는 p, q  
$\acute p_u = p_u + \eta \left(e_{(u, i)}q_i - \lambda p_u \right)$  
$\acute q_i = q_i + \eta \left(e_{(u, i)}p_u - \lambda q_i \right)$
    - $e_{(u, i)}$ : 실제 행렬 값과 예측 행렬 값의 차이로 $r_{(u, i)}-\hat r_{(u, i)}$로 계산
    - $\eta$ : SGD 학습률
    - $\lambda$ : L2 규제 계수

### 확률적 경사 하강법 구현

In [1]:
import numpy as np

# 원본 행렬 R 생성, 잠재 요인=3으로 설정
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
             [np.NaN, 5, np.NaN, 3, 1],
             [np.NaN, np.NaN, 3, 4, 4],
             [5, 2, 1, 2, np.NaN]])
num_users, num_items = R.shape
K=3

# P와 Q의 행렬 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [2]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 예측 R 행렬
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 행렬에서 null이 아닌 값의 위치 인덱스를 추출하여 실제 R과 예측 R의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [3]:
# R>0인 행(user), 열(item) 위치와 값을 저장
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j]>0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

# SGD 기법으로 P, Q 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i, :] += learning_rate * (eij * Q[j, :] - r_lambda*P[i, :])
        Q[j, :] += learning_rate * (eij * P[i, :] - r_lambda*Q[j, :])
    rmse = get_rmse(R, P, Q, non_zeros)
    if step % 50 == 0:
        print('### iteration step :', step, 'rmse :', rmse)

### iteration step : 0 rmse : 3.2388050277987723
### iteration step : 50 rmse : 0.4876723101369648
### iteration step : 100 rmse : 0.1564340384819247
### iteration step : 150 rmse : 0.07455141311978046
### iteration step : 200 rmse : 0.04325226798579314
### iteration step : 250 rmse : 0.029248328780878973
### iteration step : 300 rmse : 0.022621116143829466
### iteration step : 350 rmse : 0.019493636196525135
### iteration step : 400 rmse : 0.018022719092132704
### iteration step : 450 rmse : 0.01731968595344266
### iteration step : 500 rmse : 0.016973657887570753
### iteration step : 550 rmse : 0.016796804595895633
### iteration step : 600 rmse : 0.01670132290188466
### iteration step : 650 rmse : 0.01664473691247669
### iteration step : 700 rmse : 0.016605910068210026
### iteration step : 750 rmse : 0.016574200475705
### iteration step : 800 rmse : 0.01654431582921597
### iteration step : 850 rmse : 0.01651375177473524
### iteration step : 900 rmse : 0.01648146573819501
### iteration

In [5]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬 :', np.round(pred_matrix, 3), sep='\n')

예측 행렬 :
[[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


In [6]:
print('실제 행렬 :', np.round(R, 3), sep='\n')

실제 행렬 :
[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]
